https://www.youtube.com/watch?v=0GrciaGYzV0 - This is the video to follow to get this example.

Example. Titanic Dataset

Your first goal should be getting a generalized prediction as fast as possible.

This doesn't mean skip Exploratory Data Analysis (EDA). It just means not to get caught up in it. Don't get hung up on the hang-ups. Initially, do only what is need to get a generalized prediction. 

Getting a prediction first sets a benchmark for yourself. As you make improvement to the model, you should see imporvement to the desired error metric.

In [11]:
#With the goal above, I will import only what I need
import sklearn
from sklearn.ensemble import RandomForestRegressor

#The error metric. In this case we will use c-stat
from sklearn.metrics import roc_auc_score

#An efficient data structure
import pandas as pd

#import the data
X = pd.read_csv('train.csv')
y = X.pop("Survived")
test = pd.read_csv("test.csv")

In [2]:
X.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,3.000000,80.000000,8.000000,6.000000,512.329200


I know there are categorical variables in the data set but I will skip them for the moment. The .describe method only shows the numerical data.

In [3]:
#impute Age with the mean
X["Age"].fillna(X.Age.mean(), inplace = True)


#confirm this code is correct
X.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.836071,13.002015,1.102743,0.806057,49.693429
min,1.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,2.000000,22.000000,0.000000,0.000000,7.910400
50%,446.000000,3.000000,29.699118,0.000000,0.000000,14.454200
75%,668.500000,3.000000,35.000000,1.000000,0.000000,31.000000
max,891.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [12]:
test["Age"].fillna(test["Age"].mean(), inplace=True)
test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,418.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,12.634534,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,23.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,30.272590,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,35.750000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [13]:
test["Fare"].fillna(test["Fare"].mean(), inplace=True)
test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,12.634534,0.896760,0.981429,55.840500
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,23.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,30.272590,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,35.750000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [4]:
# Get the numeric variables by only selecting the variables that are not 'object' datatypes
numeric_variables = list(X.dtypes[X.dtypes != "object"].index)
X[numeric_variables].head()
#A good way to get a FAST model, as FAST as possible, is to ignore all the categorical variables.

,PassengerId,Pclass,Age,SibSp,Parch,Fare
0,1,3,22.0,1,0,7.2500
1,2,1,38.0,1,0,71.2833
2,3,3,26.0,0,0,7.9250
3,4,1,35.0,1,0,53.1000
4,5,3,35.0,0,0,8.0500


I notice that PassengerID looks like  worthless variable, I leave it in for two reasons. First, I dont want to go through the effort of dropping it(although that would be very easy). Second, I am interested to see if it is useful for prediction. It might be useful if Passenger ID is assigned in some non-random way. For example, perhaps Passenger ID was assigned based on when the ticket was purchased in which case there might be something predictive about if someone purchased their ticket early or late.

In [5]:
#Lets build our first model. I always have oob=True. It is a good idea to increase n_estimators to a number higher
#than the default. In this case, the oob_predictions will be based on a forest of 33 trees. I set random state=42 so
#you all replicate the model exactly.
model = RandomForestRegressor(n_estimators = 100, oob_score = True, random_state = 42)

#I only use numeric variables because I have yet to dummy out the categorical variables
model.fit(X[numeric_variables],y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=True, random_state=42, verbose=0, warm_start=False)

In [6]:
#For regression, the oob_score_ attribute gives the R^2 based on the oob predictions. We want to use c-stat, but I 
#mention that for awarenss. By the way, attributes in sklearn that have a trailing underscore are only available 
#after the model has been trained
model.oob_score_

0.1361695005913669

In [7]:
y_oob = model.oob_prediction_
print("c-stat: ",roc_auc_score(y,y_oob))

c-stat:  0.73995515504


We now have a benchmark. This isn't very good for this data set but it gives us a benchmark for improvement. Before changing parameters for the Random Forest, let's whip this dataset into shape.

In [8]:
#Here is a simple function to show descriptive stats on the categorical variables
def describe_categorical(X):
    """
    Just like .describe(), but returns the results for 
    categorical variables only.
    """
    from IPython.display import display, HTML
    display(HTML(X[X.columns[X.dtypes == "object"]].describe().to_html()))

In [9]:
describe_categorical(X)

,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Weisz, Mrs. Leopold (Mathilde Francoise Pede)",male,1601,B96 B98,S
freq,1,577,7,4,644


In [14]:
X.drop(["Name","Ticket","PassengerId"], axis = 1, inplace=True)
test.drop(["Name", "Ticket"], axis=1, inplace=True)
id = test.pop("PassengerId")

In [15]:
#Change the cabin variable to be only the first letter or None
def clean_cabin(x):
    try:
        return x[0]
    except TypeError:
        return "None"
    
X["Cabin"] = X.Cabin.apply(clean_cabin)

In [16]:
categorical_variables = ['Sex','Cabin','Embarked']

for variable in categorical_variables:
    #Fill missing data with the word missing
    X[variable].fillna("Missing", inplace=True)
    #Create array of dummies
    dummies = pd.get_dummies(X[variable], prefix = variable)
    #Update X to include dummies and drop the main interval
    X = pd.concat([X,dummies], axis = 1)
    X.drop([variable], axis=1, inplace=True)

In [17]:
X

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_None,Cabin_T,Embarked_C,Embarked_Missing,Embarked_Q,Embarked_S
0,3,22.0,1,0,7.2500,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1
1,1,38.0,1,0,71.2833,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0
2,3,26.0,0,0,7.9250,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
3,1,35.0,1,0,53.1000,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1
4,3,35.0,0,0,8.0500,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1
5,3,NaN,0,0,8.4583,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0
6,1,54.0,0,0,51.8625,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1
7,3,2.0,3,1,21.0750,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1
8,3,27.0,0,2,11.1333,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
9,2,14.0,1,0,30.0708,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0


In [18]:
#Look at all the columns in the dataset
def printall(X, max_rows=10):
    from IPython.display import display, HTML
    display(HTML(X.to_html(max_rows=max_rows)))
    
printall(X)

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_None,Cabin_T,Embarked_C,Embarked_Missing,Embarked_Q,Embarked_S
0,3,22.0,1,0,7.2500,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1
1,1,38.0,1,0,71.2833,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0
2,3,26.0,0,0,7.9250,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
3,1,35.0,1,0,53.1000,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1
4,3,35.0,0,0,8.0500,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,2,27.0,0,0,13.0000,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1
887,1,19.0,0,0,30.0000,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1
888,3,NaN,1,2,23.4500,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
889,1,26.0,0,0,30.0000,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0


There are columns that are worthless but we will keep them in for ease.

model = RandomForestRegressor(100, oob_score=True, n_jobs=-1, random_state=42)
model.fit(X,y)
print("C-stat: ", roc_auc_score(y, model.oob_prediction_))

This is a pretty good model. Now, before we try some different parameters for the model, lets use the Random Forest to help with some EDA

# Variable Importance Measures

In [19]:
model.feature_importances_

array([ 0.31538584,  0.08292152,  0.233442  ,  0.04874595,  0.03235287,
        0.28715181])

In [20]:
import matplotlib.pyplot as plt
#Simple version that shows all of the variables
feature_importances = pd.Series(model.feature_importances_,index=X.columns)
feature_importances.plot(kind='barh',figsize=(7,6))
plt.show()

ValueError: Wrong number of items passed 6, placement implies 20

In [ ]:
#Complex version that shows the summary view

# """
# By Mike Bernico
# Graphs the feature importances of a random decision forest using a horizontal bar chart.
# Probably works but untested on other sklearn.ensembles

# Parameters
# ----------

# ensemble = Name of the ensemble whose features you would like graphed.
# feature_names = A list of the names of those features, displayed on the y-axis.
# autoscale = True (Automatically adjust the x-axis size to the largest feature + .headroom) / False = scale from 0 to 1
# headroom = used with autoscale, .05 default
# width = figure width in inches
# summarized_columns = a list of column prefixes to summarize on, for dummy variables (e.g. ["day_"] would summarize all day)
#     """

def graph_feature_importances(model, feature_names, autoscale=True, headroom=0.05, width=10, summarized_columns=None):
    if autoscale:
        x_scale = model.feature_importances_.max() + headroom
    else:
        x_scale = 1
        
    feature_dict=dict(zip(feature_names, model.feature_importances_))
    
    if summarized_columns:
        #some dummy columns need to be summarized
        for col_name in summarized_columns:
            #sum all the features that contain col name, store in temp sum_value
            sum_value = sum(x for i, x in feature_dict.items() if col_name in i )
            #now remove all keys that are part of col_name
            keys_to_remove = [i for i in feature_dict.keys() if col_name in i ]
            for i in keys_to_remove:
                feature_dict.pop(i)
            #lastly, read the summarized field
            feature_dict[col_name] = sum_value
    results = pd.Series(feature_dict, index=feature_dict.keys())
    results.sort_values(inplace=True)
    print(results)
    results.plot(kind='barh', figsize=(width, len(results)/4), xlim=(0, x_scale))
    plt.show()
graph_feature_importances(model, X.columns, summarized_columns=categorical_variables)

# Parameter Tests

Parameters to test.

# Parameters that will make your model better

n_estimators: The number of trees in the forest. Choose as high a number as your computer will handle.

max_features: The number of features to consider when looking for the best split. Try["auto", "None", "sqrt", "log2", 0.9 and 0.2

min_samples_leaf: The minimum number of samples in newly created leaves. Try[1,2,3]. If 3 is the best, try higher numbers like 1-10.



# Parameters that will make it easier to train your model

n_jobs: Determines if multiple processors should be used to to train and test the model. Always set this to -1 and %%timeit vs. if it is set to 1. It should be much faster (especially when many trees are trained).

# n_jobs

In [ ]:
%% timeit
model = RandomForestRegressor(1000, oob_score=True, n_jobs=1, random_state=42)
model.fit(X,y)

In [ ]:
%% timeit
model = RandomForestRegressor(1000, oob_score=True, n_jobs=-1, random_state=42)
model.fit(X,y)

# n_estimators

In [ ]:
results = []
n_estimator_options = [30, 50, 100, 200, 500, 1000, 2000]

for trees in n_estimator_options:
    model = RandomForestRegressor(trees, oob_score=True, n_jobs=-1, random_state=42)
    model.fit(X, y)
    print(trees, 'trees')
    roc = roc_auc_score(y, model.oob_prediction_)
    print('C-stat: ', roc)
    results.append(roc)
    print (" ")
    
pd.Series(results, n_estimator_options).plot();
plt.show()

# max_features

In [ ]:
results = []
max_features_options = ["auto", None, "sqrt", "log2", 0.9, 0.2]

for max_features in max_features_options:
    model = RandomForestRegressor(n_estimators=1000, oob_score=True, n_jobs=-1, random_state=42, max_features=max_features)
    model.fit(X, y)
    print(max_features, "option")
    roc = roc_auc_score(y, model.oob_prediction_)
    print('C-stat: ', roc)
    results.append(roc)
    print (" ")
    
pd.Series(results, max_features_options).plot(kind='barh', xlim=(.85, .88))
plt.show()

# min_samples_leaf

In [ ]:
results = []
min_samples_leaf_options = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

for min_samples in min_samples_leaf_options:
    model = RandomForestRegressor(n_estimators=1000, oob_score=True, n_jobs=-1, random_state=42, max_features="auto", min_samples_leaf=min_samples)
    model.fit(X, y)
    print(min_samples, "min samples")
    roc = roc_auc_score(y, model.oob_prediction_)
    print('C-stat: ', roc)
    results.append(roc)
    print (" ")
    
pd.Series(results, min_samples_leaf_options).plot()
plt.show()

# Final Model

In [ ]:
model = RandomForestRegressor(n_estimators=1000, 
                              oob_score=True, 
                              n_jobs=-1, 
                              random_state=42, 
                              max_features="auto", 
                              min_samples_leaf=5)
model.fit(X, y)
roc = roc_auc_score(y, model.oob_prediction_)
print('C-stat: ', roc)


# Test Data

Sample Train Data

In [ ]:
test_X = X[:10]

In [ ]:
test_X

In [ ]:
from sklearn.preprocessing import StandardScaler

scaled_X = StandardScaler().fit(X[numeric_variables]).transform(X[numeric_variables])